# Results

In this notebook we explore the results of the experiments, which are stored in wandb

In [4]:
# Get wandb api object
import wandb

api = wandb.Api()

# Get all runs
project = api.project("pysentimiento")


# Get all runs for sentiment and lang es
runs = [r for r in api.runs("pysentimiento", {"config.lang": "en"}) if not r.sweep]



In [8]:
data = []

renames = {
    "vinai/bertweet-base": "bertweet",
    "roberta-base": "roberta",
    "bert-base-uncased": "bert",
    "google/electra-base-discriminator": "electra",
    "pysentimiento/robertuito-base-uncased": "robertuito",
}

odd_runs = []

for run in runs:
    # Get model name
    model_name = run.config["model"]
    # Get task 
    task = run.config["task"]
    
    # Get Macro f1 performance
    try:
        macro_f1 = run.summary["test_macro_f1"]
    except KeyError:
        print(run.name)
        odd_runs.append(run)
        continue 
    
    data.append({
        "model": model_name,
        "task": task,
        "macro_f1": macro_f1,
    })

electric-sweep-12
glowing-sweep-11
wobbly-sweep-10
clear-sweep-9
woven-sweep-8
vocal-sweep-7
rural-sweep-6
logical-sweep-5
autumn-sweep-4
cerulean-sweep-3
earnest-sweep-2
ethereal-sweep-1


In [6]:
import pandas as pd

df = pd.DataFrame(data)

df["model"] = df["model"].map(renames)
# Group by model and task

grouped = df.groupby(["task", "model"])

# Get mean macro f1 plus standard deviation in a single line with +- inbetween

df_results = (grouped.mean() * 100).round(1).astype(str) + " +- " + (grouped.std() * 100).round(1).astype(str)

df_results = df_results.unstack("task")
df_results.columns = df_results.columns.droplevel(0)

df_results

task,emotion,hate_speech,irony,sentiment
model,,,,
bert,42.7 +- 0.6,56.0 +- 0.8,68.1 +- 2.2,69.6 +- 0.4
bertweet,43.1 +- 1.8,57.7 +- 0.7,80.8 +- 0.7,72.0 +- 0.4
electra,37.2 +- 2.9,55.6 +- 0.6,71.3 +- 1.8,70.9 +- 0.4
roberta,45.0 +- 0.9,55.1 +- 0.4,70.4 +- 2.9,70.4 +- 0.3
robertuito,43.0 +- 3.3,57.5 +- 0.2,73.9 +- 1.4,69.6 +- 0.5


In [7]:
mean_df = pd.DataFrame(data).groupby(["model", "task"]).mean()
std_df = pd.DataFrame(data).groupby(["model", "task"]).std()

# Concat mean and std
df = pd.concat([mean_df, std_df], axis=1)

df.columns = ["mean macro f1", "std macro f1"]

df = df * 100

df

mean macro f1  std macro f1
model                                 task                                    
bert-base-uncased                     emotion          42.732472      0.608128
                                      hate_speech      55.966784      0.776115
                                      irony            68.110517      2.194008
                                      sentiment        69.566153      0.389755
google/electra-base-discriminator     emotion          37.195190      2.933112
                                      hate_speech      55.566145      0.590012
                                      irony            71.265484      1.829708
                                      sentiment        70.948625      0.442205
pysentimiento/robertuito-base-uncased emotion          43.036093      3.322289
                                      hate_speech      57.524815      0.240745
                                      irony            73.943002      1.398705
                                      sentiment        69.618116      0.517407
roberta-base                          emotion          45.048218      0.911606
                                      hate_speech      55.102649      0.361325
                                      irony            70.388373      2.916010
                                      sentiment        70.393631      0.305479
vinai/bertweet-base                   emotion          43.123814      1.767333
                                      hate_speech      57.727658      0.686588
                                      irony            80.766381      0.706471
                                      sentiment        72.042550      0.351604

In [5]:
df.reset_index().to_csv("../data/results_en.csv", index=False)

In [6]:
df = pd.read_csv("../data/results_en.csv").set_index(["model", "task"])

df.loc[("pysentimiento/robertuito-base-uncased", "emotion"), "mean macro f1"]

43.0360934138298